In [2]:
# make 100,000 row california dataset for testing purposes
import csv

with open ('california.csv', 'r') as object:
    csv_reader = reader(object)
    n = 0
    for row in csv_reader:
        with open('california100000.csv', 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(row)
        n += 1
        if n % 10000 == 0:
                print(n)
        if n == 100000:
            break



FileNotFoundError: [Errno 2] No such file or directory: 'california.csv'

Products

### Create long table of moves.
- Create a long intermediate dataframe for each PID to analyze moves.  Then dump period 1 locations and period 2 moves to master long table of moves.

- read fields pid, idate, odate, z4type, effdate[1-9], fips[x-9]

- write all fields WHERE effdate is period 1 or period 2.  Write a new field for which period the move took place.

How to determine move: 1) compare effdates from two moves with same PID,  2) z4type is not G (general delivery), F (firm), P (PO box).  This leaves H (high-rise, includes apartments), S (street, mostly R1), R (rural route/contract).
Write dataframe to disk (sqlite) every 10,000 records.

In [4]:
###########################
#Makes Long Table of Moves FOR ALL PERIODS#
###########################

from csv import reader
import csv
import pandas as pd
from datetime import datetime, timedelta
from itertools import islice, compress
import numpy as np

# read CSV one line at a time
keepcolumns = [0, 4, 5, 13, 14, 22, 23, 31, 32, 40,
                        41, 49, 50, 58, 59, 67, 68, 76,
                        77, 85, 86, 94, 95, 99, 103, 107, 111, 115,
                        119, 123, 127, 131, 135]

long_moves = pd.DataFrame()
z4types = ['S', 'H', 'R']


#settings
source_file = "data/all_states.csv"
status_file = "status_allmoves.txt"
start = datetime.now()
print_line_every = 500
start_line = 1
process_records = 27408937
end_line = start_line + process_records

#read status.txt for last line and ask user if they want to start with the last line
with open(status_file) as f:
    last_status_line = int(f.read())
resume_from_status = input('Type Yes to resume from last line processed in {} which is: {:,}\n'.format(status_file,last_status_line))
if resume_from_status == "Yes":
    start_line = last_status_line + 1

def period_assign(row):
    # [X] TODO: maybe use enums/categories
    if row['effdate'] >= 201301 and row['effdate'] < 201801:
        val = 2
    elif row['effdate'] >= 200801 and row['effdate'] < 201301:
        val = 1
    # [X] TODO: Add period here
    else:
        val = 0
    return val

# TODO: Make this more efficient
with open(source_file, 'r') as object:
    line_count = start_line
    process_start = datetime.now()
    csvfile = csv.reader(object)
    for row in csvfile:
        # extract the data we want into a ephemeral dataframe
        # ephemeral_row = row[keepcolumns]
        # TODO: use dask and process by chunks
        df = pd.DataFrame({'address_observation': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                                    'pid': [row[0], row[0], row[0], row[0], row[0], row[0],
                                         row[0], row[0], row[0], row[0]],
                                     'idate': [row[4], row[4], row[4], row[4], row[4], row[4],
                                         row[4], row[4], row[4], row[4]],
                                     'odate': [row[5], row[5], row[5], row[5], row[5], row[5],
                                         row[5], row[5], row[5], row[5]],
                                     'z4type': [row[13], row[22], row[31], row[40], row[49], row[58],
                                         row[67], row[76], row[85], row[94]],
                                     'effdate': [row[14], row[23], row[32], row[41], row[50], row[59],
                                         row[68], row[77], row[86], row[95]],
                                     'fips': [row[99], row[103], row[107], row[111], row[115], row[119], row[123],
                                         row[127], row[131], row[135]]})
        # if FIPS is blank, drop from dataframe.
        df = df[df.fips != '']
        # keep if z4type is S, H or R
        df = df[df['z4type'].isin(z4types)]
        # assign period values
        # [X] TODO: use read_csv(dtype=)
        df['effdate'] = pd.to_numeric(df['effdate'])
        df['idate'] = pd.to_numeric(df['idate'])
        df['odate'] = pd.to_numeric(df['odate'])
        # ~~TODO~~: Maybe pick out rows with correct period here instead of at end
        df['period'] = ""
        # ~~TODO~~: try to use swifter and apply to whole dataset/chunk
        df['period'] = df.apply(period_assign, axis=1, result_type = 'reduce')
        # remove observations seen for less than a year
        df['seentime'] = abs(df['odate'] - df['idate'])
        #df = df[df['seentime'] > 10000]
        # sort df and find origin fips
        df.sort_values(by = ['effdate'], ascending = True, inplace = True)
        # assign originfips drop second row when these are equal
        # [X] BUG: for example, if address_observation=2 has fips='' or z4type not
        #      in z4types, then address_observation=3 sets
        #      originfips=(address_observation=1).fips and
        #      prev_effdate=((address_observation=1).effdate
        # FIX: fips='' --> fips='Not in CA'
        # [X] BUG: shifts in the wrong direction
        df['originfips'] = df.fips.shift(1, fill_value = 'first record')
        df['prev_effdate'] = df.effdate.shift(1).astype(str)
        #### TO DO: SUM observations of records in the FIPS
        #drop moves before or after sample period
        #df = df[df['period'] != 0]
        # drop second row when effdates are the same. # UPDATE IT IS NOT DOING THIS BECAUSE THERE's NO INPLACE
        df.drop_duplicates(subset=['effdate'])
        #append df to long_moves table of all moves
        #drop first row when there is no move
        df.dropna(axis=0, subset = ['originfips'], how='any', inplace=True)
        long_moves = long_moves.append(df)
        line_count += 1
        if line_count % print_line_every == 0:
            # TODO: try keeping this file open throughout
            long_moves.to_csv(path_or_buf = 'data/allmoves.csv', mode='a', header=False, index=False)
            long_moves.drop(long_moves.index, inplace=True)
            process_duration = datetime.now() - process_start
            rate = print_line_every / (process_duration.total_seconds()/60)
            estimated_completion = (end_line - line_count) / rate
            print('Finished: {}, PID: {} at {} | rate: {:.1f} per min. | this batch complete: {:.1f} min'.format(line_count, row[0], datetime.now().time(), rate, estimated_completion))
            process_start = datetime.now()
            # TODO: try keeping this file open throughout
            with open(status_file, 'w') as writer:
                writer.write(str(line_count))
    # [X] BUG: last set of long_moves not written if not multiple of
    #      print_line_every

Type Yes to resume from last line processed in status_allmoves.txt which is: 74,611,500
 Yes


Finished: 74612000, PID: Y39398460047686 at 03:23:03.752107 | rate: 4807.0 per min. | this batch complete: -9819.6 min
Finished: 74612500, PID: Y39398460085811 at 03:23:09.713962 | rate: 5032.5 per min. | this batch complete: -9379.7 min
Finished: 74613000, PID: Y39398460078063 at 03:23:15.986707 | rate: 4783.0 per min. | this batch complete: -9869.0 min
Finished: 74613500, PID: Y39398460099531 at 03:23:21.736809 | rate: 5217.7 per min. | this batch complete: -9047.0 min
Finished: 74614000, PID: Y39398460003506 at 03:23:27.581548 | rate: 5133.3 per min. | this batch complete: -9195.8 min
Finished: 74614500, PID: Y39398460058807 at 03:23:33.793778 | rate: 4829.6 per min. | this batch complete: -9774.2 min
Finished: 74615000, PID: Y39398460021190 at 03:23:39.798249 | rate: 4996.5 per min. | this batch complete: -9447.9 min
Finished: 74615500, PID: Y39398460144019 at 03:23:46.083209 | rate: 4773.4 per min. | this batch complete: -9889.4 min
Finished: 74616000, PID: Y39398460187993 at 03:2

KeyboardInterrupt: 

In [83]:
#longmoves2_backup.csv (early June) is 6,652,705 rows and 609 MB
#longmoves2.csv (mid June) is 6,968,068 rows and 637 MB


In [1]:
#write header to longmoves_check
import csv
import pandas as pd

header = 'address_observation,pid,idate,odate,z4type,effdate,fips,period,seentime,originfips,prev_effdate\n'

# ~~TODO~~: maybe do this manually
with open('data/longmoves_header.csv','w') as f:
    w = f.write(header)
    f.close()

In [22]:
#write header to longmoves

import os

header = 'address_observation,pid,idate,odate,z4type,effdate,fips,period,seentime,originfips,prev_effdate\n'

def insert(originalfile,string):
    with open(originalfile,'r') as f:
        with open('data/method2/allmoves_deduped_header.csv','w', newline='\n') as f2: 
            f2.write(string)
            f2.write(f.read())
    #os.rename('longmoves2_header2.csv',originalfile)

insert('data/method2/allmoves_deduped.csv', header)

In [21]:
import pandas as pd

df = pd.read_csv('data/method2/allmoves_w_header.csv')
# [X] TODO: Double check if we only want to keep earliest residences
df.drop_duplicates(subset='pid',inplace=True)
df.to_csv('data/method2/allmoves_deduped.csv')

In [36]:
check_df = pd.read_csv('longmoves2_header2.csv')
check_df = check_df[check_df['originfips'] != 'first record']
check_df.to_csv('longmoves_check.csv', index=False)

# then compare to longmoves2_backup 

In [24]:
check_df.columns

Index(['address_observation', 'pid', 'idate', 'odate', 'z4type', 'effdate',
       'fips', 'period', 'seentime', 'originfips', 'prev_effdate'],
      dtype='object')

In [37]:
from csv_diff import load_csv, compare

diff = compare(
    load_csv(open("longmoves_check.csv")),
    load_csv(open("longmoves2_backup.csv"))
)



In [40]:
f = open('longmoves_check.csv')
f.close()

In [7]:
diff.keys()

dict_keys(['added', 'removed', 'changed', 'columns_added', 'columns_removed'])

In [50]:
removed = diff['removed']

In [46]:
import csv

with open('removed.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, diff['removed'][0].keys())
    w.writeheader()
    w.writerows(diff['removed'])


AttributeError: 'list' object has no attribute 'keys'

Assign each respondent to a period 1 tract.  Where did this person live in at the end of period 1?  Product is Count of # of respondents in that tract at the end of period 1.  This is a count per FIPS for Period 1 location.  This is done through SUMIF for each tract FIPS where period move is "1"). Analyze master long table as dataframe, result should be written as a database table.
Count respondents moving out of high-loss tracts.  Each period 2 move, regardless of destination, increments tract count by 1.   Each move out a high loss tract during period 2 counts as a separate move.   Analyze master long table as dataframe, store as database table. 
Track respondents moving out 

of high-loss tracts.  Convert master long table of moves into a wide matrix of moves where origin is a high-loss tract. Product is a matrix with origin (Y) and destination (X) tracts and values as count. Decide whether to pre-load all FIPS as rows keys or append as a new destination FIPS is incurred. Each move out a high loss tract during period 2 counts as a separate move.  This is likely a database table.
Count respondents moving into high-gain tracts.  Count of those moving into high-gain tracts. Use same script as for "Count respondents moving out of high-loss tracts" but with different values for FIPS. 
Track respondents moving into high-gain tracts.  For each person moving into a high-gain tract, was their origin FIPS 1) high-loss, 2) high-gain, or 3) other.  Create a list of dictionaries for all FIPS as keys and high-loss, high-gain, other as values. Use script for "Track respondents moving out of high-loss tracts" with lookup value for each FIPS key.  Product is a dataframe with high-gain tracts FIPS as rows and columns for counts of origin tract categories high-loss, high-gain, and other. 

In [17]:
# List of dictionaries for fips
from csv import reader

fips_dict = {}

# read csv
with open ('fips_tracts_cats.csv', 'r') as object:
    csv_reader = reader(object)
    for row in csv_reader:
        k = row[1]
        if row[5] == "1":
            v = "gain"
        elif row[6] == "1":
            v = "loss"
        else:
            v = "other"
        fips_dict[k] = v
        
# create a high gain and high loss tract list
gain_list = [k for k, v in fips_dict.items() if v == "gain"]
loss_list = [k for k, v in fips_dict.items() if v == "loss"]

In [23]:
# load dataframe
import pandas as pd

df = pd.read_csv("data/method2/allmoves_deduped.csv")

In [24]:
len(df[df['period'] == 1])

2276390

201212.0

In [25]:
len(df[df['period'] == 2])

2709873

In [ ]:
# count respondents moving into high-gain tracks

gain_dict = {}

for i in gain_list:
    df1 = df[df.fips == i]
    df2 = df1[df.period == 1]
    gain_dict[i] = len(df2)
    #print(len(df2))
    

# count respondents moving out of high-loss tracks
print('Loss List Time')
loss_dict = {}

for i in loss_list:
    df1 = df[df.originfips == i]
    df2 = df1[df.period == 1]
    loss_dict[i] = len(df2)
    #print(len(df2))


# write files to csv
import csv

# XXX: output file
with open('data/losstotal_p1.csv','w', newline='') as f:
    w = csv.writer(f)
    w.writerows(loss_dict.items())

# XXX: output file 
with open('data/gaintotal_p1.csv','w', newline='') as f:
    w = csv.writer(f)
    w.writerows(gain_dict.items())


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [ ]:
# create a matrix from period 1 moves
#prepare period 1 moves
import numpy as np
df1 = df[df.period == 1]
origins = df1.originfips.unique().tolist()
destinations = df1.fips.unique().tolist()
focusfips = loss_list + gain_list
allfips = origins + destinations
my_dict = {i:allfips.count(i) for i in allfips}
uniquefips = list(my_dict.keys())

#Create Matrix with all values as 0

matrix = pd.DataFrame(0, index=uniquefips, columns=uniquefips)

#need all fips from longmoves2

#create matrix of just period 1 moves 

for index, row in df1.iterrows():
    origin = row['originfips']
    destination = row['fips']
    matrix.loc[origin, destination] += 1

#Table with Origin Y and destination X FIPS
#increment count  for each 

# XXX: output file
matrix.to_csv("data/matrix_p1.csv")
print(matrix.shape)
print(np.count_nonzero(matrix.index.duplicated('first')))

In [ ]:
# ~~TODO~~: try getting info from pre-matrix data

# moving out of high-loss tracts during period 1
# filter on rows where index is loss_list
matrix = pd.read_csv('data/matrix_p1.csv', dtype=str, index_col=0, header=0)
#matrix.set_index('Unnamed: 0', inplace=True)

for i in uniquefips:
    matrix[i] = pd.to_numeric(matrix[i], errors='ignore')

df1 = matrix[matrix.index.isin(loss_list)]

for index, row in df1.iterrows():
    df1.loc[df1.index, df1.index] = 0

#df1.apply(pd.to_numeric, errors='coerce')

# sum of row where column is not row index value
df1totals = df1.sum(axis=1).astype(int)

#out_of_high_loss = dict(zip(df1.index, df1.total))
#write dict to csv
# XXX: output file
df1totals.to_csv('data/out_of_high_loss_p1.csv')

# moving into high-gain tracts
# pivot so that destination is rows and filter on rows where index or value is gain_list

# [X] BUG: Selects originfips that are in gain_list, then transposes
df2 = matrix[matrix.index.isin(gain_list)].transpose()

for index, row in df1.iterrows():
    df2.loc[index, index] = 0

#df2.apply(pd.to_numeric, errors='coerce')

#df2totals = pd.DataFrame()
#df2 = df2.apply(pd.to_numeric(downcast = 'integer'))
df2totals = df2.sum(axis=1).astype(int)

# for each row sum of row where column is not row index

df2totals = df2totals.to_frame()
df2totals.rename(columns={0: 'count'}, inplace=True)
df2totals['type'] = 'other'

for index, row in df2totals.iterrows():
    df2totals.at[index, 'type'] = fips_dict.get(index)

# XXX: output file
df2totals.to_csv('data/into_high_gain_p1.csv')

In [241]:
ms = matrix.index.to_series()
pd.to_numeric(ms)
ms

ValueError: Unable to parse string "Not in California" at position 191

In [ ]:
# create counts of thos moving into loss, gain, and other tracts
from_loss = np.sum(df2totals[df2totals.index.isin(loss_list)]['count'])
from_gain = np.sum(df2totals[df2totals.index.isin(gain_list)]['count'])
from_other = np.sum(df2totals['count']) - from_loss - from_gain

#from_gain = sum(into_high_gain[your_key] for your_key in gain_list.values())

dict_counts = {'from_loss': from_loss, 'from_gain': from_gain, 'from_other': from_other}
# XXX: output file
with open('data/into_high_gain_summary_p1.csv','w', newline='') as f:
    w = csv.writer(f)
    w.writerows(dict_counts.items())

In [29]:
df2totals.rename(columns={0: 'count'}, inplace=True)
np.sum(df2totals[df2totals.index.isin(loss_list)]['count'])
df2totals.sum()

count    471718.0
dtype: float64

In [ ]:
############################
#Finds counts as of 12/2012#
############################

import csv
import pandas as pd
from datetime import datetime, timedelta
from itertools import islice, compress
import numpy as np

# read CSV one line at a time
keepcolumns = [0, 4, 5, 13, 14, 22, 23, 31, 32, 40, 
                        41, 49, 50, 58, 59, 67, 68, 76, 
                        77, 85, 86, 94, 95, 99, 103, 107, 111, 115, 
                        119, 123, 127, 131, 135]

lived122012 = pd.DataFrame()
z4types = ['S', 'H', 'R']


#settings
source_file = "data/all_states.csv"
status_file = "status_counts.txt"
start = datetime.now()
print_line_every = 10000
start_line = 1
process_records = 27403370
end_line = start_line + process_records

#read status.txt for last line and ask user if they want to start with the last line
with open(status_file) as f:
    last_status_line = int(f.read())
#resume_from_status = input('Type Yes to resume from last line processed in {} which is: {:,}\n'.format(status_file,last_status_line))
#if resume_from_status == "Yes":
start_line = last_status_line + 1        

def period_assign(row):
    if row['effdate'] >= 201301 and row['effdate'] < 201801:
        val = 2
    elif row['effdate'] >= 200801 and row['effdate'] < 201301:
        val = 1
    else:
        val = 0
    return val

# TODO: bring changes from second block here
with open(source_file, 'r') as object:
    result_count = 0
    line_count = start_line
    process_start = datetime.now()
    csvfile = csv.reader(object)
    for row in csvfile:
        # extract the data we want into a ephemeral dataframe
        # ephemeral_row = row[keepcolumns]
        df = pd.DataFrame({'address_observation': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                                    'pid': [row[0], row[0], row[0], row[0], row[0], row[0],
                                         row[0], row[0], row[0], row[0]],
                                     'idate': [row[4], row[4], row[4], row[4], row[4], row[4],
                                         row[4], row[4], row[4], row[4]],
                                     'odate': [row[5], row[5], row[5], row[5], row[5], row[5],
                                         row[5], row[5], row[5], row[5]],
                                     'z4type': [row[13], row[22], row[31], row[40], row[49], row[58],
                                         row[67], row[76], row[85], row[94]],
                                     'effdate': [row[14], row[23], row[32], row[41], row[50], row[59],
                                         row[68], row[77], row[86], row[95]],
                                     'fips': [row[99], row[103], row[107], row[111], row[115], row[119], row[123],
                                         row[127], row[131], row[135]]})
        # if FIPS is blank, drop from dataframe.
        df = df[df.fips != ''] 
        
        # keep if z4type is S, H or R
        df = df[df['z4type'].isin(z4types)]
        # assign period values
        df['effdate'] = pd.to_numeric(df['effdate'], downcast='integer')
        df['idate'] = pd.to_numeric(df['idate'], downcast='integer')
        # if idate is after 201212 drop from dataframe
        df = df[df['idate'] <= 20121231]
        df['odate'] = pd.to_numeric(df['odate'])
        # if odate is before 201212 drop from dataframe     
        df = df[df['odate'] >= 20121231]
        df['period'] = ""
        #df['period'] = df.apply(period_assign, axis=1)
        # remove observations seen for less than a year
        # df['seentime'] = abs(df['odate'] - df['idate'])
        # df = df[df['seentime'] > 10000]
        # sort df and find origin fips
        df.sort_values(by = ['effdate'], ascending = True, inplace = True)
        # assign originfips drop second row when these are equal
        df['originfips'] = df.fips.shift(1)
        df['prev_effdate'] = df.effdate.shift(periods = 1, fill_value = 197001)
        df.drop_duplicates(subset=['effdate'])
        #### TO DO: SUM observations of records in the FIPS
        # if effdate is before or equal to 201212
        # and period = 0 or 
        df['next_effdate'] = df.effdate.shift(periods = -1, fill_value = 202201)
        df = df[df['effdate'] <= 201212]
        df = df[df['next_effdate'] > 201212]
        if not df.empty:            
            lived122012 = lived122012.append(df)
            result_count += 1
            #print(result_count, end=", ")
            
        #append df to long_moves table of all moves
        line_count += 1
        if line_count % print_line_every == 0:
            lived122012.to_csv(path_or_buf = 'data/lived122012.csv', mode='a', header=False, index=False)
            lived122012.drop(lived122012.index, inplace=True)
            process_duration = datetime.now() - process_start
            rate = print_line_every / (process_duration.total_seconds()/60)
            estimated_completion = (end_line - line_count) / rate
            print('Finished: {}, PID: {} at {} | rate: {:.1f} per min. | this batch complete: {:.1f} min'.format(line_count, row[0], datetime.now().time(), rate, estimated_completion)) 
            process_start = datetime.now()              
            with open(status_file, 'w') as writer:
                writer.write(str(line_count))

Finished: 10000, PID: Y39394533870912 at 18:49:15.400757 | rate: 6719.8 per min. | this batch complete: 4076.5 min
Finished: 20000, PID: Y39394203356046 at 18:51:07.619833 | rate: 5346.7 per min. | this batch complete: 5121.6 min
Finished: 30000, PID: Y39394652241028 at 18:52:53.529961 | rate: 5665.2 per min. | this batch complete: 4831.8 min
Finished: 40000, PID: Y39394652234059 at 18:54:32.744219 | rate: 6047.5 per min. | this batch complete: 4524.7 min
Finished: 50000, PID: Y39394939271530 at 18:56:01.909586 | rate: 6729.1 per min. | this batch complete: 4064.9 min
Finished: 60000, PID: Y39394572371210 at 18:57:29.434002 | rate: 6855.3 per min. | this batch complete: 3988.7 min
Finished: 70000, PID: Y39394573413103 at 18:58:54.181895 | rate: 7079.8 per min. | this batch complete: 3860.7 min
Finished: 80000, PID: Y39394316307625 at 19:00:22.033548 | rate: 6829.7 per min. | this batch complete: 4000.7 min
Finished: 90000, PID: Y39394696731152 at 19:02:09.565977 | rate: 5579.7 per min.

#6 count of where people lived at end of period 1 is not possible with my long_moves. I would have to rerun for just this question.
Likely need this as base  to compare moves with.
  
  
The whole California dataset is 26,207,392 rows long.


In [ ]:
## Count where individuals lived in 12/2012

import csv
import pandas as pd

df = pd.read_csv("data/lived122012.csv")

lived_122012_dict = {}

for i in uniquefips:
    df1 = df[df.fips == i]
    lived_122012_dict[i] = len(df1)
    print(len(df2))

# write files to csv

with open('data/lived_122012_totals.csv','w', newline='') as f:
    w = csv.writer(f)
    w.writerows(lived_122012_dict.items())

In [ ]:
############################
#Finds counts as of 12/2012 USING PANDAS!!!!#
############################

import csv
import pandas as pd
from datetime import datetime, timedelta
from itertools import islice, compress
import numpy as np

# read CSV one line at a time
keepcolumns = [0, 4, 5, 13, 14, 22, 23, 31, 32, 40, 
                        41, 49, 50, 58, 59, 67, 68, 76, 
                        77, 85, 86, 94, 95, 99, 103, 107, 111, 115, 
                        119, 123, 127, 131, 135]

lived122012 = pd.DataFrame()
z4types = ['S', 'H', 'R']


#settings
source_file = "data/allstates_deduped.csv"
status_file = "status_counts.txt"
start = datetime.now()
print_line_every = 10000
start_line = 1
process_records = 27403370
end_line = start_line + process_records

#read status.txt for last line and ask user if they want to start with the last line
with open(status_file) as f:
    last_status_line = int(f.read())
#resume_from_status = input('Type Yes to resume from last line processed in {} which is: {:,}\n'.format(status_file,last_status_line))
#if resume_from_status == "Yes":
start_line = last_status_line + 1        

def period_assign(row):
    if row['effdate'] >= 201301 and row['effdate'] < 201801:
        val = 2
    elif row['effdate'] >= 200801 and row['effdate'] < 201301:
        val = 1
    else:
        val = 0
    return val

# TODO: bring changes down here too
with open(source_file, 'r') as object:
    result_count = 0
    line_count = start_line
    process_start = datetime.now()
    csvfile = csv.reader(object)
    for row in csvfile:
        # extract the data we want into a ephemeral dataframe
        # ephemeral_row = row[keepcolumns]
        df = pd.DataFrame({'address_observation': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                                    'pid': [row[0], row[0], row[0], row[0], row[0], row[0],
                                         row[0], row[0], row[0], row[0]],
                                     'idate': [row[4], row[4], row[4], row[4], row[4], row[4],
                                         row[4], row[4], row[4], row[4]],
                                     'odate': [row[5], row[5], row[5], row[5], row[5], row[5],
                                         row[5], row[5], row[5], row[5]],
                                     'z4type': [row[13], row[22], row[31], row[40], row[49], row[58],
                                         row[67], row[76], row[85], row[94]],
                                     'effdate': [row[14], row[23], row[32], row[41], row[50], row[59],
                                         row[68], row[77], row[86], row[95]],
                                     'fips': [row[99], row[103], row[107], row[111], row[115], row[119], row[123],
                                         row[127], row[131], row[135]]})
        # if FIPS is blank, drop from dataframe.
        df = df[df.fips != ''] 
        
        # keep if z4type is S, H or R
        df = df[df['z4type'].isin(z4types)]
        # assign period values
        df['effdate'] = pd.to_numeric(df['effdate'], downcast='integer')
        df['idate'] = pd.to_numeric(df['idate'], downcast='integer')
        # if idate is after 201212 drop from dataframe
        df = df[df['idate'] <= 20121231]
        df['odate'] = pd.to_numeric(df['odate'])
        # if odate is before 201212 drop from dataframe     
        df = df[df['odate'] >= 20121231]
        df['period'] = ""
        #df['period'] = df.apply(period_assign, axis=1)
        # remove observations seen for less than a year
        # df['seentime'] = abs(df['odate'] - df['idate'])
        # df = df[df['seentime'] > 10000]
        # sort df and find origin fips
        df.sort_values(by = ['effdate'], ascending = True, inplace = True)
        # assign originfips drop second row when these are equal
        df['originfips'] = df.fips.shift(1)
        df['prev_effdate'] = df.effdate.shift(periods = 1, fill_value = 197001)
        df.drop_duplicates(subset=['effdate'])
        #### TO DO: SUM observations of records in the FIPS
        # if effdate is before or equal to 201212
        # and period = 0 or 
        df['next_effdate'] = df.effdate.shift(periods = -1, fill_value = 202201)
        df = df[df['effdate'] <= 201212]
        df = df[df['next_effdate'] > 201212]
        if not df.empty:            
            lived122012 = lived122012.append(df)
            result_count += 1
            #print(result_count, end=", ")
            
        #append df to long_moves table of all moves
        line_count += 1
        if line_count % print_line_every == 0:
            lived122012.to_csv(path_or_buf = 'data/lived122012.csv', mode='a', header=False, index=False)
            lived122012.drop(lived122012.index, inplace=True)
            process_duration = datetime.now() - process_start
            rate = print_line_every / (process_duration.total_seconds()/60)
            estimated_completion = (end_line - line_count) / rate
            print('Finished: {}, PID: {} at {} | rate: {:.1f} per min. | this batch complete: {:.1f} min'.format(line_count, row[0], datetime.now().time(), rate, estimated_completion)) 
            process_start = datetime.now()              
            with open(status_file, 'w') as writer:
                writer.write(str(line_count))